In [2]:
print("ok")

ok


In [3]:
%pwd

'g:\\finalchatbot\\HealthChatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'g:\\finalchatbot\\HealthChatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Adobe InDesign CS4 (6.0)', 'creationdate': '2015-12-10T00:43:23+00:00', 'title': 'Food for Fitness', 'author': 'Bean, Anita', 'moddate': '2015-12-13T22:58:55-08:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 274, 'page': 0, 'page_label': 'Cover'}, page_content='Anita Bean is a registered nutritionist (Sports & \nExercise) whose practical style has made her one \nof the UK’s most respected nutrition writers. She \nis an accomplished sportsperson and author of the \nbestselling titles: The Complete Guide to Sports \nNutrition and The Complete Guide to Strength \nTraining. Winner of two major achievement awards \nin sports nutrition she features regularly in the media, \nand contributes to numerous health and fitness \npublications. \nA sports nutrition guide and recipe book rolled \ninto one, Food for Fitness dispels popular myths \nand gives you the tools you need to reach your \nmaximum performance. \nFood fo

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1490


In [ ]:
#text_chunks

[Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Adobe InDesign CS4 (6.0)', 'creationdate': '2015-12-10T00:43:23+00:00', 'title': 'Food for Fitness', 'author': 'Bean, Anita', 'moddate': '2015-12-13T22:58:55-08:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 274, 'page': 0, 'page_label': 'Cover'}, page_content='Anita Bean is a registered nutritionist (Sports & \nExercise) whose practical style has made her one \nof the UK’s most respected nutrition writers. She \nis an accomplished sportsperson and author of the \nbestselling titles: The Complete Guide to Sports \nNutrition and The Complete Guide to Strength \nTraining. Winner of two major achievement awards \nin sports nutrition she features regularly in the media, \nand contributes to numerous health and fitness \npublications.'),
 Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Adobe InDesign CS4 (6.0)', 'creationdate': '2015-12-10T00:43:23+00:00', 'title': 'Food for Fitness', 'author':

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:

embeddings = download_hugging_face_embeddings()

c:\Users\Dell\anaconda3\envs\healthbot\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os


pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [65]:
import os
from dotenv import load_dotenv

load_dotenv()

print("Pinecone API Key:", os.getenv("PINECONE_API_KEY"))

Pinecone API Key: pcsk_6uxWhp_QboBvY1q8Mw4S3u39uJZwrHgPsEaKEoKM2fgxQLust1AJz6miE5AHpGJDbkGRM6


In [70]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Fetch API keys safely
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Ensure they are not None before setting them
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is missing. Check your .env file.")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is missing. Check your .env file.")

# Set environment variables manually
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("API Keys Loaded Successfully!")


API Keys Loaded Successfully!


In [67]:

# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [34]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [37]:

retrieved_docs = retriever.invoke(" Where do I get energy?")

In [38]:

retrieved_docs

[Document(id='e0b8f4d3-9c4c-4126-be51-1f6c881dabc6', metadata={'author': 'Bean, Anita', 'creationdate': '2015-12-10T00:43:23+00:00', 'creator': 'Adobe InDesign CS4 (6.0)', 'moddate': '2015-12-13T22:58:55-08:00', 'page': 10.0, 'page_label': '3', 'producer': 'Adobe PDF Library 9.0', 'source': 'Data\\Medical_book.pdf', 'title': 'Food for Fitness', 'total_pages': 274.0}, page_content='nutrition essentials\n3\n1 NUTRITION ESSENTIALS\n1. Energy (calories)\nWhere do I get energy?\nYou get energy from four components in food and drink: carbohydrate, fat, \nprotein and (though not strictly a nutrient) alcohol. Each of these provides \ndifferent amounts of energy. 1 g provides approximately:\n protein  4 kcal (17 kJ) alcohol  7 kcal (29 kJ)\n carbohydrate  4 kcal (17 kJ) fat  9 kcal (38 kJ)\nThese fuels are broken down in your body and transformed via various bio\xad'),
 Document(id='8bba55f1-ee32-4f7b-abdc-5f91ab5668fc', metadata={'author': 'Bean, Anita', 'creationdate': '2015-12-10T00:43:23+00

In [ ]:
#llm here 
from langchain_openai import OpenAI
#llm = OpenAI(temperature=0.4, max_tokens=500)
llm = OpenAI(
    model_name="text-davinci-003",
    max_tokens=500,  # Limit response length
    temperature=0.7,  # Reduce randomness
)


In [51]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [52]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [71]:

response = rag_chain.invoke({"input": " Where do I get energy?"})
print(response["answer"])



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}